In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas import DataFrame
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import collections

pd.options.display.max_rows=999

pd.options.display.max_columns=999

In [2]:
#function to lag

def buildLaggedFeatures(s,columns, lag=5,dropna=True): #lag=3
    '''
    From http://stackoverflow.com/questions/20410312/how-to-create-a-lagged-data-structure-using-pandas-dataframe
    Builds a new DataFrame to facilitate regressing over all possible lagged features
    '''
    if type(s) is pd.DataFrame:
        new_dict={}
        for c in s.columns:
            new_dict[c]=s[c]
        for col_name in columns:
            new_dict[col_name]=s[col_name]
            # create lagged Series
            for l in range(1,lag+1):
                new_dict['%s_lag%d' %(col_name,l)]=s[col_name].shift(l)
        res=pd.DataFrame(new_dict,index=s.index)

    elif type(s) is pd.Series:
        the_range=range(lag+1)
        res=pd.concat([s.shift(i) for i in the_range],axis=1)
        res.columns=['lag_%d' %i for i in the_range]
    else:
        print('Only works for DataFrame or Series')
        return None
    if dropna:
        return res.dropna()
    else:
        return res 


#function to set in each row if there is an event in the 4h before 
    
def make_time_before_ev(df,ev,event_1h_bef,event_2h_bef,event_3h_bef,event_4h_bef,event_1h_aft,event_2h_aft,event_3h_aft,event_4h_aft,
                       l_topic_0,l_topic_1,l_topic_2,l_topic_3,l_topic_4,l_topic_5,l_topic_6,l_topic_7,
                       Time_difference):
    
    print("---------------------------------------------------")
    print(len(df))
    
    df.reset_index(drop=True, inplace=True)

    for dw_index, dw_row in df.iterrows():
        if dw_index%5000==0:
            print(dw_index)
                
        for e_index, e_row in ev.iterrows():
           
            if e_row.starting_time.day == dw_row.ObservedArrivalDateTime.day and e_row.starting_time.month == dw_row.ObservedArrivalDateTime.month and e_row.starting_time.year == dw_row.ObservedArrivalDateTime.year:
                
                diff= e_row.starting_time - dw_row.ObservedArrivalDateTime
                diff = diff.total_seconds() / 3600 

                if diff <=4 and diff > -4:
                    l_topic_0[dw_index]=e_row.topic_0
                    l_topic_1[dw_index]=e_row.topic_1
                    l_topic_2[dw_index]=e_row.topic_2
                    l_topic_3[dw_index]=e_row.topic_3
                    l_topic_4[dw_index]=e_row.topic_4
                    l_topic_5[dw_index]=e_row.topic_5
                    l_topic_6[dw_index]=e_row.topic_6
                    l_topic_7[dw_index]=e_row.topic_7

                if Time_difference[dw_index]>diff:
                    Time_difference[dw_index]=diff

                if diff > 0 and diff <= 1:
                    event_1h_bef[dw_index] = 1
                        
                elif diff > 1 and diff <= 2:
                    event_2h_bef[dw_index] = 1

                elif diff > 2 and diff <= 3:
                    event_3h_bef[dw_index] = 1

                elif diff > 3 and diff <= 4:
                    event_4h_bef[dw_index] = 1

                elif diff > -1 and diff <= 0:
                    event_1h_aft[dw_index] = 1

                elif diff > -2 and diff <= -1:
                    event_2h_aft[dw_index] = 1

                elif diff > -3 and diff <= -2:
                    event_3h_aft[dw_index] = 1

                elif diff > -4 and diff <= -3:
                    event_4h_aft[dw_index] = 1
                    
                    
    list_topics.append(l_topic_0)
    list_topics.append(l_topic_1)
    list_topics.append(l_topic_2)
    list_topics.append(l_topic_3)
    list_topics.append(l_topic_4)
    list_topics.append(l_topic_5)
    list_topics.append(l_topic_6)
    list_topics.append(l_topic_7)


In [3]:
#opening the stops dataframe
stop_point = pd.read_csv("Files/Poi_StopPoint_1km.csv", sep=';',decimal=',')
venue_poi=stop_point['PoiName'].unique()
stop_point=stop_point[['PoiName', 'StopPointId', 'Distance']]
stop_point=stop_point[stop_point['StopPointId']<8000000]

# selecting the closest stops
list_closest_stops=[]
n=3          #number of closest stops to consider
for ven in venue_poi:
    if ven == 'Bella Center' or ven =='Royal Arena':
        n=8
    else:
        n=4
        
    only_one=stop_point[stop_point['PoiName']==ven]
    only_one.sort_values(by='Distance',inplace=True)
    for i in range(n):
        list_closest_stops.append(only_one.iloc[i])

list_closest_stops_df=pd.DataFrame(list_closest_stops)



C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
events17=pd.read_csv("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\events\\READY\\Event_with_topics_label.csv",encoding='latin-1'      )
events17['starting_time']=pd.to_datetime(events17['starting_time'],format="%Y-%m-%d %H:%M:%S")
events17['end_time']=pd.to_datetime(events17['end_time'],format="%Y-%m-%d %H:%M:%S")


In [5]:
# opening dwelTimes datasets
# file_names_2016=['Poi_Dwell_1km_201601',	"Poi_Dwell_1km_201602",	"Poi_Dwell_1km_201603",	"Poi_Dwell_1km_201604",	"Poi_Dwell_1km_201605",	"Poi_Dwell_1km_201606",	"Poi_Dwell_1km_201607",	"Poi_Dwell_1km_201608",	"Poi_Dwell_1km_201609",	"Poi_Dwell_1km_201610",	"Poi_Dwell_1km_201611",	"Poi_Dwell_1km_201612"]
file_names_2017=[	"Poi_Dwell_1km_201701",	"Poi_Dwell_1km_201702",	"Poi_Dwell_1km_201703",	"Poi_Dwell_1km_201704",	"Poi_Dwell_1km_201705",	"Poi_Dwell_1km_201706",	"Poi_Dwell_1km_201707",	"Poi_Dwell_1km_201708",	"Poi_Dwell_1km_201709",	"Poi_Dwell_1km_201710",	"Poi_Dwell_1km_201711",	"Poi_Dwell_1km_201712"]

dw_list=[]

# for name in file_names_2016:
#     dwellTime_file = pd.read_csv("Files/DwellTimes/2016/%s.csv" %(name), sep=',', usecols=['ObservedArrivalDateTime','DwellSeconds', 'StopPointId',"IsFirstStop","IsLastStop","IsTimingPoint"] ,
#     names = ["OperatingDayDate", "ObservedArrivalDateTime", "ObservedDepartureDateTime", "LineDirectionCode",
#     "StopPointId","IsFirstStop","IsLastStop","IsTimingPoint","DwellSeconds","JourneyRef","JourneyPointRef"])
#     dw_list.append(dwellTime_file)

for name in file_names_2017:
    dwellTime_file = pd.read_csv("Files/DwellTimes/2017/%s.csv" %(name), sep=',', usecols=['ObservedArrivalDateTime','DwellSeconds', 'StopPointId',"IsFirstStop","IsLastStop","IsTimingPoint"] ,
    names = ["OperatingDayDate", "ObservedArrivalDateTime", "ObservedDepartureDateTime", "LineDirectionCode",
    "StopPointId","IsFirstStop","IsLastStop","IsTimingPoint","DwellSeconds","JourneyRef","JourneyPointRef"])
    dw_list.append(dwellTime_file)

dwellTime= pd.concat(dw_list)

dwellTime['ObservedArrivalDateTime']=pd.to_datetime(dwellTime['ObservedArrivalDateTime'], format="%Y-%m-%d %H:%M:%S")

#cleaning from unneccessary data 
dwellTime.dropna(subset=['DwellSeconds'],inplace=True)
dwellTime=dwellTime[dwellTime['IsFirstStop']==0]
dwellTime=dwellTime[dwellTime['IsLastStop']==0]
dwellTime=dwellTime[dwellTime['IsTimingPoint']==0]
dwellTime.drop(['IsFirstStop', 'IsLastStop','IsTimingPoint' ], axis=1, inplace=True)
dwellTime.reset_index(drop=True, inplace=True)

#create a df that considers only the closest stops for each venue using the dataframe crated before
df_dw_merged=pd.merge(dwellTime, list_closest_stops_df, how='outer', on='StopPointId')
df_dw_merged_small=df_dw_merged.dropna(subset=['PoiName'])
df_dw_merged_small_dw=df_dw_merged_small.dropna(subset=['DwellSeconds'])
df_dw_merged_small_dw.reset_index(drop=True,inplace=True)

#diveide the df by year
# df_2016=df_dw_merged_small_dw[df_dw_merged_small_dw['ObservedArrivalDateTime'].dt.year==2016]
df_2017=df_dw_merged_small_dw[df_dw_merged_small_dw['ObservedArrivalDateTime'].dt.year==2017]

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:


######SELECT
venue_names=['Parken', 'Vega', 'Forum', 'Bella Center', 'Royal Arena','DR Koncerthuset']
venue_names2= [ 'TeliaParken', 'Vega', 'ForumCopenhagen','BellaCenter',  'RoyalArena','DrKoncerthuset']

#create df of event per each venue
list_df_per_venue=[]
for ven_nam in venue_names:
    df_per_ven=df_2017[df_2017['PoiName']==ven_nam]
    list_df_per_venue.append(df_per_ven)

In [16]:
list_df_per_venue[4].StopPointId.unique()

array([30943, 30941, 30944, 30942], dtype=int64)

In [26]:
df_2017[df_2017['StopPointId']==30944]

,ObservedArrivalDateTime,StopPointId,DwellSeconds,PoiName,Distance
720907,2017-01-01 06:50:06,30944,0.0,Royal Arena,709.262505
720908,2017-01-01 07:50:42,30944,0.0,Royal Arena,709.262505
720909,2017-01-01 08:10:46,30944,0.0,Royal Arena,709.262505
720910,2017-01-01 08:48:41,30944,0.0,Royal Arena,709.262505
720911,2017-01-01 09:33:25,30944,0.0,Royal Arena,709.262505
720912,2017-01-01 10:08:16,30944,0.0,Royal Arena,709.262505
720913,2017-01-01 10:37:18,30944,0.0,Royal Arena,709.262505
720914,2017-01-01 11:07:45,30944,0.0,Royal Arena,709.262505
720915,2017-01-01 11:38:27,30944,0.0,Royal Arena,709.262505
720916,2017-01-01 12:06:46,30944,0.0,Royal Arena,709.262505


In [15]:
list_df_per_venue[4][list_df_per_venue[4]['StopPointId']==list_df_per_venue[4].StopPointId.unique()[0]]

,ObservedArrivalDateTime,StopPointId,DwellSeconds,PoiName,Distance
587936,2017-01-01 06:00:45,30943,0.0,Royal Arena,710.128964
587937,2017-01-01 06:58:54,30943,0.0,Royal Arena,710.128964
587938,2017-01-01 07:58:55,30943,0.0,Royal Arena,710.128964
587939,2017-01-01 08:54:46,30943,0.0,Royal Arena,710.128964
587940,2017-01-01 09:56:24,30943,0.0,Royal Arena,710.128964
587941,2017-01-01 10:08:13,30943,0.0,Royal Arena,710.128964
587942,2017-01-01 10:39:59,30943,0.0,Royal Arena,710.128964
587943,2017-01-01 10:55:25,30943,0.0,Royal Arena,710.128964
587944,2017-01-01 11:09:18,30943,0.0,Royal Arena,710.128964
587945,2017-01-01 11:37:20,30943,0.0,Royal Arena,710.128964


# Only 5 venues

In [15]:
######SELECT
venue_names=['Parken', 'Vega', 'Forum', 'Bella Center', 'Royal Arena','DR Koncerthuset']
venue_names2= [ 'TeliaParken', 'Vega', 'ForumCopenhagen','BellaCenter',  'RoyalArena','DrKoncerthuset']

# 2017

In [22]:
topics_array=events17[['Music', 'Alternative&Punk_Music', 'Business', 'Children',
       'Classic_Music', 'Classical_Music', 'Culture', 'Design', 'Education',
       'Electronica_Music', 'Entertainment', 'Football', 'Fælledparken',
       'Indipendent_Music', 'Lifestyle', 'Medicine', 'Metal_Music', 'Music.1',
       'Pop_Music', 'Religious_Music', 'Rock_Music', 'Sport',
       'StageMusicals_Music', 'Traditional_Music', 'Urban_Music',
       'entertainment']].values

In [27]:
#create df of event per each venue
list_df_per_venue=[]
for ven_nam in venue_names:
    df_per_ven=df_2017[df_2017['PoiName']==ven_nam]
    list_df_per_venue.append(df_per_ven)

#create ev of event per each venue
list_ev_per_venue=[]
for ven_nam in venue_names2:
    ev_per_ven=events17[events17['venue_unique']==ven_nam]
    list_ev_per_venue.append(ev_per_ven)
    
#use the event dataset to check when there are the event
df_ult_list=[]

for i in range(6):
    df=list_df_per_venue[i]
    ev=list_ev_per_venue[i]
    
    event_1h_bef= np.zeros((len(df,)))
    event_2h_bef= np.zeros((len(df,)))
    event_3h_bef= np.zeros((len(df,)))

    event_during= np.zeros((len(df,)))
    
    event_1h_aft= np.zeros((len(df,)))
    event_2h_aft= np.zeros((len(df,)))
    event_3h_aft= np.zeros((len(df,)))
    
    Time_difference=[9999999999 for x in range(len(df))]
    
    topic=np.zeros((len(df,),26))
    
    print("---------------------------------------------------")
    print(len(df))
    
    df.reset_index(drop=True, inplace=True)

    for dw_index, dw_row in df.iterrows():
        if dw_index%5000==0:
            print(dw_index)
                
        for e_index, e_row in ev.iterrows():
           
            if e_row.starting_time.day == dw_row.ObservedArrivalDateTime.day and e_row.starting_time.month == dw_row.ObservedArrivalDateTime.month and e_row.starting_time.year == dw_row.ObservedArrivalDateTime.year:
                
                diff= e_row.starting_time - dw_row.ObservedArrivalDateTime
                diff = diff.total_seconds() / 3600 

                if diff >=-3 and diff <= 3:
                    topic[dw_index]=topics_array[e_index]

 
                if Time_difference[dw_index]>diff:
                    Time_difference[dw_index]=diff

                if diff > 0 and diff <= 1:
                    event_1h_bef[dw_index] = 1

                if diff > 1 and diff <= 2:
                    event_2h_bef[dw_index] = 1

                if diff > 2 and diff <= 3:
                    event_3h_bef[dw_index] = 1

                diff= e_row.end_time - dw_row.ObservedArrivalDateTime
                diff = diff.total_seconds() / 3600 
                
                
                if diff <= 0 and diff >=-1:
                    event_1h_aft[dw_index] = 1
                    
                if diff < -1 and diff >=-2:
                    event_2h_aft[dw_index] = 1
                    
                if diff < -2 and diff >=-3:
                    event_3h_aft[dw_index] = 1
                
                if dw_row.ObservedArrivalDateTime>e_row.starting_time and dw_row.ObservedArrivalDateTime<e_row.end_time:
                    event_during[dw_index]=1
                    topic[dw_index]=topics_array[e_index]


    df['event_1h_bef']=event_1h_bef
    df['event_2h_bef']=event_2h_bef
    df['event_3h_bef']=event_3h_bef

    df['event_1h_aft']=event_1h_aft
    df['event_2h_aft']=event_2h_aft
    df['event_3h_aft']=event_3h_aft

    
    df['Time_difference']=Time_difference
    
    df['event_during']=event_during
    
    topicDF=pd.DataFrame(topic,columns=[['Music', 'Alternative&Punk_Music', 'Business', 'Children',
       'Classic_Music', 'Classical_Music', 'Culture', 'Design', 'Education',
       'Electronica_Music', 'Entertainment', 'Football', 'Fælledparken',
       'Indipendent_Music', 'Lifestyle', 'Medicine', 'Metal_Music', 'Music.1',
       'Pop_Music', 'Religious_Music', 'Rock_Music', 'Sport',
       'StageMusicals_Music', 'Traditional_Music', 'Urban_Music',
       'entertainment']])

    df_t=pd.concat([df,topicDF],axis=1)

    
    df_ult_list.append(df_t)

---------------------------------------------------
84719
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

---------------------------------------------------
224747
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
---------------------------------------------------
190238
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
---------------------------------------------------
134987
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
---------------------------------------------------
29911
0
5000
10000
15000
20000
25000
--------------------------------

In [28]:
#lag dwell second by stop
df_all=pd.concat(df_ult_list)
list_stopID=df_all.StopPointId.unique()
list_df_per_stop=[]
for stopID in list_stopID:
    df_per_stop=df_all[df_all['StopPointId']==stopID]
    list_df_per_stop.append(df_per_stop)

list_df_lagged=[]
for df in list_df_per_stop:
    df_lagged=buildLaggedFeatures(df, ['DwellSeconds'])
    list_df_lagged.append(df_lagged)

#put togetther 
df_final=pd.concat(list_df_lagged)
df_final.sort_values(by='ObservedArrivalDateTime',inplace=True)
df_final.reset_index(drop=True, inplace=True)

#datatime
df_final["day"] = df_final["ObservedArrivalDateTime"].dt.weekday
df_final["hours"] = df_final["ObservedArrivalDateTime"].dt.hour
ho=np.array(pd.get_dummies(df_final["hours"]))
ho=pd.DataFrame(ho)
df_final=pd.concat([df_final, ho], axis=1)
df_final['isWeekend']=df_final['day'].apply(lambda x: x>4)
df_final['isWeekend']=df_final['isWeekend']*1

#rename column of hour
dfs_new=[]
dfs_new.append(df_final)
for i in range(24):
    df_old=dfs_new[i]
    df_new=df_old.rename(index=str, columns={i: "h_%s"%(i)})
    dfs_new.append(df_new)
    
fin_df=dfs_new[24]


#save
fin_df.to_pickle("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Dwell_2017")

In [2]:
# Dwell time dataset
f1 = open("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Dwell_2017", 'rb')
fin_df = pickle.load(f1)
f1.close()

In [24]:
col=["Temp. (Â°C)high","Temp. (Â°C)avg","Temp. (Â°C)low","Dew Point (Â°C)high","Dew Point (Â°C)avg","Dew Point (Â°C)low","Humidity (%)high",
"Humidity (%)avg","Humidity (%)low","Sea Level Press. (hPa)high","Sea Level Press. (hPa)avg","Sea Level Press. (hPa)low","Visibility (km)high",
"Visibility (km)avg","Visibility (km)low","Wind (km/h)high","Wind (km/h)avg","Wind (km/h)high","Precip. (mm)","Events"]

# weather=[]
# for i in range(12):
#     i=i+1
#     w=pd.read_csv("C:/Users/User/Desktop/DTU/Thesis/Python Files/Files/Weather data/w%s.csv"%(i), names=col,skiprows=[0,1])
#     weathers.append(w)
    
weathers_list=[]



weathers=[]
for i in range(12):
    i=i+1
    w=pd.read_csv("C:/Users/User/Desktop/DTU/Thesis/Python Files/Files/Weather data/w%s.csv"%(i), names=col,skiprows=[0,1])
    weathers.append(w)


mo=1
for df in weathers:
    df.reset_index(inplace=True)
    df['date']=pd.Timestamp('2017-%s-01'%(mo))
    mo=mo+1
    lista=[]
    for i in range(len(df)):
        d=df["index"].iloc[i]-1
        lista.append(df.date.iloc[i]+pd.Timedelta("%s days"%(d)))
    df['DateTime']=lista
    weathers_list.append(df)    
    
weather=pd.concat(weathers_list)
weather=weather[['DateTime','Temp. (Â°C)avg','Precip. (mm)']]

In [25]:
fin_df.reset_index(inplace=True,drop=True)
fin_df['Precipitation']=None
fin_df['Temperature']=None

In [26]:
for ind,row in weather.iterrows():
    month=row.DateTime.month
    day=row.DateTime.day
    fin_df['Precipitation'][(fin_df['ObservedArrivalDateTime'].dt.month==month) & (fin_df['ObservedArrivalDateTime'].dt.day==day) ]=row['Precip. (mm)'] 
    fin_df['Temperature'][(fin_df['ObservedArrivalDateTime'].dt.month==month) & (fin_df['ObservedArrivalDateTime'].dt.day==day) ]=row['Temp. (Â°C)avg']        
    if day==1:
        print(month)


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


1
2
3
4
5
6
7
8
9
10
11
12


In [27]:
#save
fin_df.to_pickle("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs\\Dwell_2017_w")

In [ ]:
# for dw_index, dw_row in df.iterrows():
#         if dw_index%5000==0:
#             print(dw_index)
                
#         for e_index, e_row in ev.iterrows():
           
#             if e_row.starting_time.day == dw_row.ObservedArrivalDateTime.day and e_row.starting_time.month == dw_row.ObservedArrivalDateTime.month and e_row.starting_time.year == dw_row.ObservedArrivalDateTime.year:
                
#                 diff= e_row.starting_time - dw_row.ObservedArrivalDateTime
#                 diff = diff.total_seconds() / 3600 


# 2016

In [ ]:
venue_poi=['BellaCenter', "DrKoncerthuset", "ForumCopenhagen",  "TeliaPerken","RoyalArena", "Vega"]

list_df_per_venue=[]
for ven_nam in venue_poi:
    df_per_ven=df_2016[df_2016['PoiName']==ven_nam]
    list_df_per_venue.append(df_per_ven)

In [ ]:
df_ult_list_2=[]
for i in range(6):
    df=list_df_per_venue[i]
    ev=ev_no0_list_2016[i]
    
    event_1h_bef= np.zeros((len(df,)))
    event_2h_bef= np.zeros((len(df,)))
    event_3h_bef= np.zeros((len(df,)))
    event_4h_bef= np.zeros((len(df,)))
    event_1h_aft= np.zeros((len(df,)))
    event_2h_aft= np.zeros((len(df,)))
    event_3h_aft= np.zeros((len(df,)))
    event_4h_aft= np.zeros((len(df,)))

    make_time_before_ev(df,ev,event_1h_bef,event_2h_bef,event_3h_bef,event_4h_bef,event_1h_aft,event_2h_aft,event_3h_aft,event_4h_aft)

    df['event_1h_bef']=event_1h_bef
    df['event_2h_bef']=event_2h_bef
    df['event_3h_bef']=event_3h_bef
    df['event_4h_bef']=event_4h_bef
    df['event_1h_aft']=event_1h_aft
    df['event_2h_aft']=event_2h_aft
    df['event_3h_aft']=event_3h_aft
    df['event_4h_aft']=event_4h_aft
    
    df_ult_list_2.append(df)

In [ ]:
#lag dwell second by stop
df_all=pd.concat(df_ult_list_2)
list_stopID=df_all.StopPointId.unique()
list_df_per_stop=[]
for stopID in list_stopID:
    df_per_stop=df_all[df_all['StopPointId']==stopID]
    list_df_per_stop.append(df_per_stop)

list_df_lagged=[]
for df in list_df_per_stop:
    df_lagged=buildLaggedFeatures(df, ['DwellSeconds'])
    list_df_lagged.append(df_lagged)

#put togetther 
df_final=pd.concat(list_df_lagged)
df_final.sort_values(by='ObservedArrivalDateTime',inplace=True)
df_final.reset_index(drop=True, inplace=True)

#datatime
df_final["day"] = df_final["ObservedArrivalDateTime"].dt.weekday
df_final["hours"] = df_final["ObservedArrivalDateTime"].dt.hour
ho=np.array(pd.get_dummies(df_final["hours"]))
ho=pd.DataFrame(ho)
df_final=pd.concat([df_final, ho], axis=1)
df_final['isWeekend']=df_final['day'].apply(lambda x: x>4)
df_final['isWeekend']=df_final['isWeekend']*1
#rename column of hour
dfs_new=[]
dfs_new.append(df_final)
for i in range(24):
    df_old=dfs_new[i]
    df_new=df_old.rename(index=str, columns={i: "h_%s"%(i)})
    dfs_new.append(df_new)
    
fin_df=dfs_new[24]

#distance betweeen venue and the nearest train station
fin_df['Distance_venue_trainSt']=0
fin_df['Distance_venue_trainSt'][fin_df['PoiName']=="Forum"]=1
fin_df['Distance_venue_trainSt'][fin_df['PoiName']=="Bella Center"]=260
fin_df['Distance_venue_trainSt'][fin_df['PoiName']=="Vega"]=850
fin_df['Distance_venue_trainSt'][fin_df['PoiName']=="Parken"]=1500
fin_df['Distance_venue_trainSt'][fin_df['PoiName']=="DR Koncerthuset"]=230
fin_df['Distance_venue_trainSt'][fin_df['PoiName']=="Royal Arena"]=531

#save
df_final.to_pickle("C:\\Users\\User\\Desktop\\DTU\\Thesis\\Python Files\\Files\\FinalDFs_DwellTimes\\All_2016")